In [1]:
import os
import csv
import pandas as pd
import datetime

In [2]:
# Defining directory where files are stored
LZ_HOME = os.path.expanduser('~')
file_dir = os.path.join(LZ_HOME, 'LZ Projects/python_exercises/exercises/03 dates_and_times/')
print(file_dir)

/Users/ludovica/LZ Projects/python_exercises/exercises/03 dates_and_times/


In [3]:
filename_csv = os.path.join(file_dir, "NY_rodent_inspections_sample_medium.csv")
print(filename_csv)

/Users/ludovica/LZ Projects/python_exercises/exercises/03 dates_and_times/NY_rodent_inspections_sample_medium.csv


In [4]:
inspections_df = pd.read_csv(filename_csv)
inspections_df.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION
0,BAIT,1,PO12965,3,1011470035,1,1147,35,104,WEST 76 STREET,10023,990505,223527,40.780204,-73.977414,Manhattan,10/14/2009 12:00:27 PM,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)"
1,BAIT,2,PO12966,3,1011470034,1,1147,34,102,WEST 76 STREET,10023,990516,223521,40.780188,-73.977375,Manhattan,10/14/2009 12:51:21 PM,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)"
2,BAIT,30,PO16966,3,2043370027,2,4337,27,620,THWAITES PLACE,10467,1020110,252216,40.858877,-73.870364,Bronx,11/09/2009 12:59:55 PM,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)"
3,BAIT,31,PO13665,3,2037670077,2,3767,77,1227,WHITEPLAINS ROAD,10472,1022441,242180,40.831321,-73.861994,Bronx,11/09/2009 11:10:16 AM,Bait applied,11/10/2009 02:56:42 PM,"(40.8313209626148, -73.861994089899)"
4,BAIT,38,PO11291,3,1011690057,1,1169,57,2199,BROADWAY,10024,989641,224567,40.783059,-73.980533,Manhattan,11/10/2009 08:40:42 AM,Bait applied,11/17/2009 11:39:11 AM,"(40.7830590725833, -73.9805333640688)"


In [5]:
format_descriptor = '%m/%d/%Y %I:%M:%S %p'

In [6]:
#Adding a column with the date in the format above
inspections_df['inspection_datetime'] = inspections_df.apply(lambda row: datetime.datetime.strptime(row['INSPECTION_DATE'], format_descriptor), axis=1)

#We add a column containing the weekday the inspection took place. To do this we read the datetime
#above  and turn it into a string that contains the full weekday name (%A)
inspections_df['inspection_weekday'] = inspections_df.apply(lambda row: row['inspection_datetime'].strftime("%A"), axis=1)

#Similar to the above but we add the full name of the month
inspections_df['inspection_month'] = inspections_df.apply(lambda row: row['inspection_datetime'].strftime("%B"), axis=1)


def format_iso_week(datetime_obj):
    the_date = datetime_obj.date()
    iso_year, iso_week, iso_weekday = the_date.isocalendar()
    iso_week_str = str(iso_year) + '-W' + str(iso_week)
    return iso_week_str

inspections_df['isoweek'] = inspections_df.apply(lambda row: format_iso_week(row['inspection_datetime']), axis=1)

#Turning the approved date into readable format

inspections_df['approved_datetime'] = inspections_df.apply(lambda row: datetime.datetime.strptime(row['APPROVED_DATE'], format_descriptor), axis=1)

#function to calculate difference between approval and inspection

def time_difference_days(date1, date2):
    difference = date2 - date1
    diff_string = str(int(difference.seconds/3600)) + " hours"
    return diff_string
    
inspections_df['time for approval'] = inspections_df.apply(lambda row: time_difference_days(row['inspection_datetime'], row['approved_datetime']), axis = 1)

In [7]:
#Function to calculate the season of inspection
autumn_ls = ["September","October","November"]
def season(date):
    if date == "December" or "January" or "February":
        return "Winter"
    elif date == "March" or "April" or "May":
        return "Spring"
    elif date == "June" or "July" or "August":
        return "Summer"
    elif date in autumn_ls:
        return "Autumn"

inspections_df['season'] = inspections_df.apply(lambda row: season(row['inspection_month']), axis=1)

In [8]:
inspections_df.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,RESULT,APPROVED_DATE,LOCATION,inspection_datetime,inspection_weekday,inspection_month,isoweek,approved_datetime,time for approval,season
0,BAIT,1,PO12965,3,1011470035,1,1147,35,104,WEST 76 STREET,...,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)",2009-10-14 12:00:27,Wednesday,October,2009-W42,2009-10-14 15:01:46,3 hours,Winter
1,BAIT,2,PO12966,3,1011470034,1,1147,34,102,WEST 76 STREET,...,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)",2009-10-14 12:51:21,Wednesday,October,2009-W42,2009-10-14 15:02:30,2 hours,Winter
2,BAIT,30,PO16966,3,2043370027,2,4337,27,620,THWAITES PLACE,...,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)",2009-11-09 12:59:55,Monday,November,2009-W46,2009-11-10 14:54:52,1 hours,Winter
3,BAIT,31,PO13665,3,2037670077,2,3767,77,1227,WHITEPLAINS ROAD,...,Bait applied,11/10/2009 02:56:42 PM,"(40.8313209626148, -73.861994089899)",2009-11-09 11:10:16,Monday,November,2009-W46,2009-11-10 14:56:42,3 hours,Winter
4,BAIT,38,PO11291,3,1011690057,1,1169,57,2199,BROADWAY,...,Bait applied,11/17/2009 11:39:11 AM,"(40.7830590725833, -73.9805333640688)",2009-11-10 08:40:42,Tuesday,November,2009-W46,2009-11-17 11:39:11,2 hours,Winter
